In [ ]:
!pip install gensim nltk

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
import os
import io

In [ ]:
df = pd.read_csv("./Data/patient_review_data.csv")

In [ ]:
df = pd.read_csv("./Data/test_data.csv")

In [ ]:
#https://github.com/anirudhshenoy/text-classification-small-datasets/tree/master/datasets

df = pd.read_csv('book_title_test_data.csv')
# Keep only the first 500 rows in df
df = df.iloc[:500]

# Rename 'title' column to 'Comment'
df.rename(columns={'title': 'Comment'}, inplace=True)
df['Comment']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      15 Highly Important Questions About Adulthood,...
1      250 Nuns Just Cycled All The Way From Kathmand...
2      Australian comedians "could have been shot" du...
3      Lycos launches screensaver to increase spammer...
4      Fußball-Bundesliga 2008–09: Goalkeeper Butt si...
                             ...                        
495    '90s Cartoons With Human Eyes May Ruin Your Ch...
496    Wait, WTF Is "I Saw Mommy Kissing Santa Claus"...
497           First Women Win Seats in Kuwait Parliament
498                   Were You A Scene Kid Or An Emo Kid
499    Opposition leader takes early lead in Sierra L...
Name: Comment, Length: 500, dtype: object

In [ ]:
# NEW CODE: LDA

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora

# Download stopwords
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Tokenize, remove stopwords and lemmatize
    return [lemmatizer.lemmatize(word) for word in gensim.utils.simple_preprocess(text) if word not in stop_words]

# Preprocess the column
processed_docs = df['Comment'].map(preprocess)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Creating the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(processed_docs)

# Converting list of documents (corpus) into Document-Term Matrix using the dictionary
doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_docs]


In [ ]:
# Creating the LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix,
                                            id2word=dictionary,
                                            num_topics=3, # Change the number of topics
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)


In [ ]:
# View the topics in LDA model
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.026*"care" + 0.022*"staff" + 0.021*"patient" + 0.018*"made" + 0.017*"hospital" + 0.015*"delay" + 0.014*"like" + 0.012*"problem" + 0.011*"feel" + 0.011*"professional"
Topic: 1 
Words: 0.034*"staff" + 0.028*"patient" + 0.016*"concern" + 0.015*"hospital" + 0.011*"abuse" + 0.011*"felt" + 0.011*"cleanliness" + 0.010*"great" + 0.010*"member" + 0.009*"room"
Topic: 2 
Words: 0.055*"time" + 0.034*"waiting" + 0.030*"care" + 0.028*"good" + 0.022*"doctor" + 0.022*"hospital" + 0.019*"staff" + 0.018*"wait" + 0.014*"room" + 0.014*"experience"


In [ ]:
!pip install gensim

In [ ]:
top_n = 5  # Set the number of top words you want to consider for each topic

top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words = [word for word, prop in lda_model.show_topic(t, topn=top_n)]
    top_words_per_topic.append(top_words)

# Optionally, print the top words for each topic
for idx, topic_words in enumerate(top_words_per_topic):
    print(f"Topic {idx}: {', '.join(topic_words)}")

Topic 0: care, staff, patient, made, hospital
Topic 1: staff, patient, concern, hospital, abuse
Topic 2: time, waiting, care, good, doctor


In [ ]:
from gensim.models.coherencemodel import CoherenceModel

# Compute Coherence Score using the top N words of each topic
coherence_model_lda = CoherenceModel(model=lda_model, topics=top_words_per_topic, dictionary=dictionary, texts=processed_docs, coherence='c_v')
#coherence_model_lda = CoherenceModel(model=lda_model, corpus=doc_term_matrix, topics=top_words_per_topic, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Coherence Score:  0.5152452116001003
